# Preparación de un conjunto de datos para el ajuste fino de instrucciones supervisadas

En esta sección , se descargará y formateará un conjunto de datos de instrucciones pasra el ajuste fino de instrucciones de un LLM preentrenado.  El  conjunto  de  datos  consta  de  1100  pares  instrucción respuesta. 

El  siguiente  código  implementa  y  ejecuta  una  función para  descargar  este  conjunto  de  datos,  un  archivo  relativamente  pequeño  (solo  204  KB),  en  formato  JSON. JSON  (Notación  de  Objetos  JavaScript)  refleja  la  estructura  de  los  diccionarios  de  Python,  proporcionando  una  estructura  sencilla  para  el  intercambio  de  datos,  legible  tanto  para  humanos  como  para  máquinas.

In [4]:
import json
import os
import urllib

def download_and_load_file(file_path, url):
    #Si no existe el archivo JSON descargado, se procede a su descarga
    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)

    #En este punto, el archivo ya existe: lo abrimos y cargamos su contenido JSON
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data

file_path = "instruction-data.json"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch07/01_main-chapter-code/instruction-data.json"
data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

Number of entries: 1100


In [5]:
print("Example entry:\n", data[50])

Example entry:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


Las  entradas  de  ejemplo  son  objetos  de  diccionario  de  Python  que  contienen  una instrucción,  una  entrada  y  una  salida. 

En  función  del  contenido  de  esta  entrada,  es  posible  que  en  ocasiones  el  campo  'entrada' esté  vacío.

In [6]:
print("Another example:\n", data[999])

Another example:
 {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}


El  ajuste  fino  de  instrucciones,  también  conocido  como  ajuste  fino  de  instrucciones  supervisadas,  implica  entrenar  un  modelo  en  un  conjunto  de  datos  donde  los  pares  de  entradasalida,  como  los  que  se extrageron  del  archivo  JSON,  se  proporcionan  explícitamente.  Existen  varios  métodos  para  formatear  estas  entradas  para  los  LLM.

La  Figura  ilustra  dos  formatos  de  ejemplo  diferentes,  a  menudo  denominados  estilos  de  indicaciones,  utilizados  en  la  formación  de  importantes  LLM  como  Alpaca  y  Phi3.  Alpaca fue  uno  de  los  primeros  LLM  en  detallar  públicamente  su  proceso  de  ajuste  de  instrucción. Phi3,  desarrollado  por  Microsoft,  se  incluye  para  demostrar  la  diversidad  de  estilos  de  indicaciones

![Texto alternativo](./imgs/7.4.png)

El  resto  de  esta sección se utiliza  el  estilo  de  indicaciones  Alpaca,  ya  que  es  uno  de  los  más  populares,  en  gran  medida  porque  ayudó  a  definir  el  enfoque  original  del  ajuste  fino.

Definir una función formar_input que se pueda usar para convertir las entradas de la lista de datos al formato de entrada estilo Alpaca.


In [7]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    return instruction_text + input_text

Esta  función  format_input  toma  una  entrada  de  diccionario  como  entrada  y  construye  una  cadena  formateada.  

In [8]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


In [9]:
model_input = format_input(data[999])
desired_response = f"\n\n### Response:\n{data[50]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is an antonym of 'complicated'?

### Response:
The correct spelling is 'Occasion.'


Como se puede ver la salida anterior, las entradas con un campo 'input' vacío no contienen una sección ###Input

Antes de configurar los cargadores de datos en Pytorch, se deve dividir el conjunto de datos en un conjunto de entrenamiento, validación y prueba.

In [ ]:
#85% entrenamiento, 10% test y 5% validacion

import pandas as pd

#Convertir la lista en DataFrame
df = pd.DataFrame(data)

# Calcular índices
train_portion = int(len(df) * 0.85)
test_portion = int(len(df) * 0.1)
val_portion = len(df) - train_portion - test_portion

#Divisiones
train_df = df.iloc[:train_portion]
test_df = df.iloc[train_portion:train_portion + test_portion]
val_df = df.iloc[train_portion + test_portion:]

#Mostrar tamaños
print("Training set length:", len(train_df))
print("Validation set length:", len(val_df))
print("Test set length:", len(test_df))

#Guardar como JSON (o CSV si prefieres)
train_df.to_json("train.json", orient="records", lines=False, force_ascii=False, indent=4)
val_df.to_json("val.json", orient="records", lines=False, force_ascii=False, indent=4)
test_df.to_json("test.json", orient="records", lines=False, force_ascii=False, indent=4)

print("\nArchivos guardados como JSON con pandas")


Training set length: 935
Validation set length: 55
Test set length: 110

✅ Archivos guardados como JSON con pandas


Tras descargar y particionar el conjunto de datos, es momento de la implentación principal del proceso de ajuste fino de instrucciones.


[Organización de datos en lotes de entrenamiento](./3_organizacion_datos_lotes_entrenamiento.ipynb)